In [ ]:
import os
import datetime
import pandas as pd
import numpy as np

from keras.initializers import he_uniform
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
from keras.regularizers import l2
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint, TensorBoard

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

seed=11
np.random.seed(seed)

## 0. Preps

In [ ]:
def sample_random_predictions(X, y):
    I = np.random.permutation(X.shape[0])[:3]

    for c, i in enumerate(I):
        img = X[i].reshape(28, 28)
        plt.subplot(131+c)
        plt.imshow(img, cmap=plt.cm.gray)
        plt.title(y[i])

In [ ]:
def print_training_history(history):
    acc_list = history.history['acc']
    acc = acc_list[-1]
    best_acc_index = np.array(acc_list).argmax()
    best_acc = acc_list[best_acc_index]
    
    print('Accuracy: {:.4f} \tBest Accuracy: {:.4f} \t\t@ {} epoch'.format(acc, best_acc, best_acc_index+1))
    
    if 'val_acc' in  history.history.keys():
        val_acc_list = history.history['val_acc']
        val_acc = val_acc_list[-1]
        best_val_acc_index = np.array(val_acc_list).argmax()
        best_val_acc = val_acc_list[best_val_acc_index]
        print('Dev Accuracy: {:.4f} \tBest Dev Accuracy: {:.4f} \t@ {} epoch'.format(val_acc, best_val_acc, best_val_acc_index+1))
    

In [ ]:
def plot_training_history(history):
    plt.figure(figsize=(15,4))

    plt.subplot(121)
    plt.plot(history.history['acc'], label='Training Set')
    
    if 'val_acc' in history.history.keys():
        plt.plot(history.history['val_acc'], label='Validation Set')
    plt.title('Accuracy vs Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(loc='best')

    plt.subplot(122)
    plt.plot(history.history['loss'], label='Training Set')
    
    if 'val_loss' in history.history.keys():
        plt.plot(history.history['val_loss'], label='Validation Set')
    plt.title('Loss vs Epochs')
    plt.xlabel('Loss')
    plt.ylabel('Accuracy')
    plt.legend(loc='best')
    plt.show()

In [ ]:
def plot_error_matrix(cm):
    row_sum = cm.sum(axis=1, keepdims=True)
    norm_cm = cm/row_sum
    
    np.fill_diagonal(norm_cm, 0)
    sns.heatmap(norm_cm, robust=True, fmt="f", cmap='RdBu_r', vmin=0, vmax=4)

    plt.show()

## 1. Getting Data

In [ ]:
payload_dir = '../payload/'
payload_file = 'normal-payload.npz'

payload_path = os.path.join(payload_dir, payload_file)

payload_archive = np.load(payload_path)

dev_set = payload_archive['dev_set']
test_set = payload_archive['test_set']
train_dev_set = payload_archive['train_dev_set']
train_set = payload_archive['train_set']

1.1 Training Set

In [ ]:
trainX, trainy = train_set[:,1:], train_set[:,0]
trainY = np_utils.to_categorical(trainy)

In [ ]:
train_devX, train_devy = train_dev_set[:,1:], train_dev_set[:,0]
train_devY = np_utils.to_categorical(train_devy)

In [ ]:
testX, testy = test_set[:,1:], test_set[:,0]
testY = np_utils.to_categorical(testy)

In [ ]:
devX, devy = dev_set[:,1:], dev_set[:,0]
devY = np_utils.to_categorical(devy)

## 4 The Network

### 4.1 Network Architecture

Regularization Parameters

In [ ]:
dropout = .2
lambd = 0

### 4.2 Optimization Setup

4.2.1 Hyperparameters

In [ ]:
alpha = 1e-4
lr_decay = 1e-3
batch_size=128
loss = 'categorical_crossentropy'
metrics = ['accuracy']

models_dir = '../models'
modelpath = os.path.join(models_dir,'MLPNet.{epoch:02d}-{val_loss:.2f}.hdf5')

4.2.2 Callbacks

In [ ]:
patience=10

In [ ]:
def lr_sched(epoch):
    return 1/(1+lr_decay*epoch) * alpha#*(lr_decay**epoch)

schd = LearningRateScheduler(lr_sched, verbose=1)
early_stopping = EarlyStopping(patience=patience, verbose=1)
model_checkpoint = ModelCheckpoint(filepath=modelpath, save_best_only=True, verbose=1, monitor='val_acc')
tfboard = TensorBoard(log_dir='./logs', batch_size=128, write_graph=True, write_images=True)


callbacks = [schd, early_stopping, model_checkpoint, tfboard]

4.2.3 The Model

In [ ]:
model = None#'MLPNet.28-0.08.hdf5'

if model:
    alpha = 0.004998650364401612
    model = os.path.join('../models', model)

In [ ]:
def get_model(path=None):        
    
    model = Sequential()
    
    if path:
        return load_model(path)
    
    # layer 1
    model.add(
        Dense(1024, 
        bias_initializer='zeros', kernel_initializer=he_uniform(seed), 
        kernel_regularizer=l2(lambd),
        input_dim=784)
    )
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    # layer 1
    model.add(
        Dense(1024, 
        bias_initializer='zeros', kernel_initializer=he_uniform(seed), 
        kernel_regularizer=l2(lambd),
        )
    )
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dense(10, activation='softmax'))
    
    adam_optimizer = Adam(lr=alpha)
    model.compile(optimizer=adam_optimizer, loss=loss, metrics=metrics)
    
    return model

In [ ]:
model = get_model(model)

4.2.4 Optimization

In [ ]:
num_epoch=200

In [ ]:
history=model.fit(trainX, trainY, batch_size=batch_size, epochs=num_epoch, verbose=1, 
                  callbacks=callbacks, validation_split=.1)

4.2.3 Best Model Yet

In [ ]:
print_training_history(history)

### 4.3 Training Result

In [ ]:
plot_training_history(history)

